# Setup Bokeh

In [1]:
import os,sys,logging, urllib

# this may be dangerous, only for local testing/debugging
os.environ["BOKEH_ALLOW_WS_ORIGIN"]="*"

# if you want more debug info
os.environ["BOKEH_LOG_LEVEL"]="debug" 

import bokeh
import bokeh.io 
import bokeh.io.notebook 
import bokeh.models.widgets  
import bokeh.core.validation
import bokeh.plotting
import bokeh.core.validation.warnings
import bokeh.layouts

bokeh.core.validation.silence(bokeh.core.validation.warnings.EMPTY_LAYOUT, True)
bokeh.io.output_notebook()

# in JypyterLab/JupyterHub we need to tell what is the proxy url
# see https://docs.bokeh.org/en/3.0.3/docs/user_guide/output/jupyter.html
def RemoteJupyterProxyUrl(port):
    if port is None:
        ret="jupyter01.classe.cornell.edu"
    else:
        ret=f"https://jupyter01.classe.cornell.edu{os.environ['JUPYTERHUB_SERVICE_PREFIX']}proxy/{port}"
    print(ret)
    return ret

def ShowApp(app, ssh_tunnels=None,notebook_url=None):
    if ssh_tunnels:
        notebook_port,bokeh_port=ssh_tunnels
        bokeh.io.notebook.show_app(app, bokeh.io.notebook.curstate(), f"http://127.0.0.1:{notebook_port}", port = bokeh_port) 
    else:
        bokeh.io.show(app)

Loading BokehJS ...

# Test if Bokeh is working

In [2]:
def MyApp(doc):
    fig = bokeh.plotting.figure(title="Multiple line example", x_axis_label="x", y_axis_label="y",height=200)
    fig.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5])
    grid=bokeh.layouts.grid(children=[fig], nrows=1, ncols=1, sizing_mode='stretch_width') 
    main_layout=bokeh.layouts.column(children=[],sizing_mode='stretch_width')
    main_layout.children.append(grid)  
    button = bokeh.models.widgets.Button(label="Bokeh is working", sizing_mode='stretch_width')
    button.on_click(lambda: print("Bokeh on_click event is working"))     
    main_layout.children.append(button)
    doc.add_root(main_layout)  
ShowApp(MyApp)

# OpenVisus

In [3]:
sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")
import OpenVisus as  ov
from OpenVisus.dashboards import Slice,Slices
os.environ["VISUS_NETSERVICE_VERBOSE"]="1"
print("OpenVisus imported")

OpenVisus imported


In [4]:
idx_filename='http://atlantis.sci.utah.edu/mod_visus?dataset=rabbit&cached=1'
db=ov.LoadDataset(idx_filename)   
print(f"logic_box={db.getLogicBox()}") 
field=db.getField()
print(f"field name={field.name} dtype={field.dtype.toString()}")

logic_box=([0, 0], [131072, 131072])
field name=EM dtype=uint8


In [5]:
def MyApp(doc):
    slice=Slice(doc,sizing_mode='stretch_width')
    slice.show_options=["palette","timestep","field","direction","offset"]
    slice.setDataset(db, direction=2)   
    slice.setPalette("Greys256", palette_range=(0,255))
    slice.setTimestep(db.getTime())
    slice.setField(field.name)         
    doc.add_root(slice.layout)
ShowApp(MyApp)